In [11]:
import json
from matplotlib import pyplot as plt
import pandas as pd

# Convergence

In [40]:

with open("../simulator/bin/simData/qt_1_n50-i1/exec_numMotes_50.dat.json") as file:
    data = json.load(file)

    for mote in data['0'].keys():
        if 'trickle' in data['0'][mote]:
            # if int(mote) == 9:
            v = data['0'][mote]['trickle']
            df = pd.DataFrame(v).T
            
            # df.to_csv('exp3.csv', index=False)
            if 'pfree' in df.columns and 'epsilon' in df.columns and 'k' in df.columns and 'preset' in df.columns:
                fig = plt.figure(figsize=(15,5))
                # plt.plot(df['pfree']*10, label='pfree', linestyle='--', marker='o')
                # plt.plot(df['epsilon']*10, label='epsilon', linestyle='--', marker='o')
                plt.plot(df['k'], label='k', linestyle='--', marker='o')
                plt.plot(df['nbr'], label='nbr', linestyle='--', marker='o')
                plt.plot(df['preset']*10, label='preset', linestyle='--', marker='o')
                plt.plot(df['counter'], label='counter', linestyle='--', marker='o')
                # plt.plot(df['t_start'], label='t_start')
                # plt.plot(df['t_end'], label='t_end')
                plt.legend()
                plt.savefig(f"figures/{mote}.png")
                plt.close()

# Grouped bar

In [13]:
measured_metrics = {
    'joining-time': 'mean',
    'current-consumed': 'mean',
    'last_all_ops': 'sum',
    'last_curr_mbr': 'mean'
}

df_ = pd.read_csv("../simulator/bin/simData/comparison_merged.csv")
c_ = df_.columns.to_list()
c_.remove('parameter')
c_.remove('method')

df_t = df_.pivot(index='parameter', columns='method', values=c_)
df_t = df_t.reset_index()
df_t.columns = df_t.columns.map('-'.join)
df_t['n'] = df_t['parameter-'].str.replace("(", "").str.replace(")", "").str.split(", ").str[0].astype(int)
df_t['i'] = df_t['parameter-'].str.replace("(", "").str.replace(")", "").str.split(", ").str[1].astype(int)
df_t.sort_values(by=['n', 'i'], inplace=True)

for c in df_t.columns:
    method = c.split("-")[-1]
    if len(method) > 1:
        new_col_name = method + "-" + c.replace(f"-{method}", "")
        df_t.rename(columns={c: new_col_name}, inplace=True)

target_ = list(measured_metrics.keys())

for col in target_:

    temp_dic = {}
    for i in df_t.columns[df_t.columns.str.contains(col)]:
        if 'ORI' in i:
            id_ = 10
        elif 'RIATA' in i:
            id_ = 20
        elif 'AC' in i:
            id_ = 30
        else:
            id_ = 40
        if "-std" in i: id_ += 1
        temp_dic[i] = id_

    sorted_cols = list(dict(sorted(temp_dic.items(), key=lambda item: item[1])).keys())
    cols = ["parameter-"] + sorted_cols
    dftt = df_t[cols]
    subkey = measured_metrics[col]
    dftt.to_csv(f"exp3/{col}-{subkey}.csv", index=False)

/Users/dzakyzf/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/Users/dzakyzf/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  app.launch_new_instance()
